# ============================================================
# FASE 3: PROCESO ETL CON PANDAS
# ============================================================
# En esta fase vamos a:
# 1. Extraer: Leer el dataset limpio
# 2. Transformar: Crear tablas dimensionales y de hechos
# 3. Cargar: Guardar en SQLite (warehouse_pandas.db)
# ============================================================


In [ ]:
# ============================================================
# PASO 1: EXTRACCIÓN (E) - Cargar dataset limpio
# ============================================================
# Cargamos el dataset que limpiamos anteriormente
import sqlite3
from sqlalchemy import create_engine

print("=" * 60)
print("PASO 1: EXTRACCIÓN (E)")
print("=" * 60)

# Cargar el dataset limpio
data_path = "data/gym_lifters_clean.csv"
df_clean = pd.read_csv(data_path)

print(f"✓ Dataset limpio cargado: {len(df_clean)} filas")
print(f"✓ Columnas: {len(df_clean.columns)}")
print("\nPrimeras 3 filas:")
display(df_clean.head(3))


## PASO 2: TRANSFORMACIÓN (T) - Crear Modelo Dimensional

Vamos a crear:
- **dim_athlete**: Información única de cada atleta
- **dim_competition**: Información de las competencias  
- **dim_team**: Información de equipos y coaches
- **fact_lifting**: Tabla de hechos con métricas de levantamiento


In [ ]:
# ============================================================
# PASO 2: TRANSFORMACIÓN (T)
# ============================================================
print("\n" + "=" * 60)
print("PASO 2: TRANSFORMACIÓN (T)")
print("=" * 60)

# ============================================================
# DIMENSIÓN 1: dim_athlete
# ============================================================
print("\n2.1 Creando dim_athlete...")

# Seleccionar columnas de atleta y eliminar duplicados
dim_athlete = df_clean[['athlete_id', 'name', 'gender', 'age', 'country']].copy()

# Filtrar filas con nombre válido
dim_athlete = dim_athlete[dim_athlete['name'].notna() & (dim_athlete['name'] != '')]

# Eliminar duplicados (mantener el primero)
dim_athlete = dim_athlete.drop_duplicates(subset=['name', 'country'], keep='first')

# Crear ID numérico único para cada atleta
dim_athlete = dim_athlete.reset_index(drop=True)
dim_athlete.insert(0, 'id_athlete', range(1, len(dim_athlete) + 1))

# Limpiar athlete_id: si está vacío, generar uno nuevo
dim_athlete['athlete_id'] = dim_athlete.apply(
    lambda row: row['athlete_id'] if pd.notna(row['athlete_id']) and row['athlete_id'] != '' 
    else f"ath_{row['id_athlete']}", axis=1
)

# Seleccionar columnas finales en el orden correcto
dim_athlete = dim_athlete[['id_athlete', 'athlete_id', 'name', 'gender', 'age', 'country']]

print(f"   ✓ dim_athlete creada: {len(dim_athlete)} atletas únicos")
print(f"   ✓ Columnas: {list(dim_athlete.columns)}")
display(dim_athlete.head(5))


In [ ]:
# ============================================================
# DIMENSIÓN 2: dim_competition
# ============================================================
print("\n2.2 Creando dim_competition...")

# Seleccionar columnas de competencia y eliminar duplicados
dim_competition = df_clean[['competition', 'year', 'category']].copy()

# Filtrar filas con competencia y año válidos
dim_competition = dim_competition[
    dim_competition['competition'].notna() & 
    dim_competition['year'].notna()
]

# Eliminar duplicados
dim_competition = dim_competition.drop_duplicates(subset=['competition', 'year', 'category'], keep='first')

# Crear ID numérico único para cada competencia
dim_competition = dim_competition.reset_index(drop=True)
dim_competition.insert(0, 'id_competition', range(1, len(dim_competition) + 1))

# Seleccionar columnas finales
dim_competition = dim_competition[['id_competition', 'competition', 'year', 'category']]

print(f"   ✓ dim_competition creada: {len(dim_competition)} competencias únicas")
print(f"   ✓ Columnas: {list(dim_competition.columns)}")
display(dim_competition.head(5))


In [ ]:
# ============================================================
# DIMENSIÓN 3: dim_team
# ============================================================
print("\n2.3 Creando dim_team...")

# Seleccionar columnas de equipo y eliminar duplicados
dim_team = df_clean[['team', 'coach']].copy()

# Filtrar filas con team válido
dim_team = dim_team[dim_team['team'].notna() & (dim_team['team'] != '')]

# Eliminar duplicados
dim_team = dim_team.drop_duplicates(subset=['team', 'coach'], keep='first')

# Crear ID numérico único para cada equipo
dim_team = dim_team.reset_index(drop=True)
dim_team.insert(0, 'id_team', range(1, len(dim_team) + 1))

# Seleccionar columnas finales
dim_team = dim_team[['id_team', 'team', 'coach']]

print(f"   ✓ dim_team creada: {len(dim_team)} equipos únicos")
print(f"   ✓ Columnas: {list(dim_team.columns)}")
display(dim_team.head(5))


In [ ]:
# ============================================================
# TABLA DE HECHOS: fact_lifting
# ============================================================
# La tabla de hechos contiene las métricas de levantamiento
# y se relaciona con las 3 dimensiones mediante claves foráneas
print("\n2.4 Creando fact_lifting (Tabla de Hechos)...")
print("-" * 60)

# Preparar el DataFrame base con todos los datos limpios
fact_base = df_clean.copy()
print(f"   Registros originales: {len(fact_base)}")

# ============================================================
# PASO 1: Relacionar con dim_athlete
# ============================================================
# Usamos merge (equivalente a JOIN en SQL) para relacionar
# los datos con la dimensión de atletas
print("\n   Paso 1: Relacionando con dim_athlete...")
fact_with_athlete = fact_base.merge(
    dim_athlete[['id_athlete', 'name', 'country']],  # Solo las columnas necesarias
    on=['name', 'country'],  # Clave de unión: nombre y país
    how='left'  # Left join: mantiene todos los registros de fact_base
)

# Contar cuántos registros encontraron match
matches_athlete = fact_with_athlete['id_athlete'].notna().sum()
print(f"      Registros con atleta encontrado: {matches_athlete} ({matches_athlete/len(fact_base)*100:.1f}%)")

# ============================================================
# PASO 2: Relacionar con dim_competition
# ============================================================
print("\n   Paso 2: Relacionando con dim_competition...")
fact_with_competition = fact_with_athlete.merge(
    dim_competition[['id_competition', 'competition', 'year', 'category']],
    on=['competition', 'year', 'category'],  # Clave de unión: competencia, año y categoría
    how='left'
)

matches_competition = fact_with_competition['id_competition'].notna().sum()
print(f"      Registros con competencia encontrada: {matches_competition} ({matches_competition/len(fact_base)*100:.1f}%)")

# ============================================================
# PASO 3: Relacionar con dim_team
# ============================================================
print("\n   Paso 3: Relacionando con dim_team...")
fact_with_team = fact_with_competition.merge(
    dim_team[['id_team', 'team', 'coach']],
    on=['team', 'coach'],  # Clave de unión: equipo y coach
    how='left'
)

matches_team = fact_with_team['id_team'].notna().sum()
print(f"      Registros con equipo encontrado: {matches_team} ({matches_team/len(fact_base)*100:.1f}%)")

# ============================================================
# PASO 4: Filtrar registros válidos
# ============================================================
# Solo mantenemos los registros que tienen todas las claves foráneas
# (id_athlete, id_competition, id_team) válidas
print("\n   Paso 4: Filtrando registros válidos...")
fact_lifting = fact_with_team[
    fact_with_team['id_athlete'].notna() &      # Debe tener atleta
    fact_with_team['id_competition'].notna() &  # Debe tener competencia
    fact_with_team['id_team'].notna()           # Debe tener equipo
].copy()

print(f"      Registros válidos (con todas las relaciones): {len(fact_lifting)}")

# ============================================================
# PASO 5: Seleccionar columnas finales
# ============================================================
# La tabla de hechos contiene:
# - Claves foráneas (id_athlete, id_competition, id_team)
# - Métricas de levantamiento (pesos, rankings, medallas, etc.)
print("\n   Paso 5: Seleccionando columnas finales...")
fact_lifting = fact_lifting[[
    # Claves foráneas (relaciones con dimensiones)
    'id_athlete',        # Relación con dim_athlete
    'id_competition',    # Relación con dim_competition
    'id_team',           # Relación con dim_team
    
    # Métricas de levantamiento
    'snatch_kg',         # Peso levantado en snatch (kg)
    'clean_and_jerk_kg', # Peso levantado en clean and jerk (kg)
    'total_kg',          # Total levantado (kg)
    'body_weight_kg',    # Peso corporal del atleta (kg)
    'event_rank',        # Posición en la competencia
    
    # Información adicional
    'medal',             # Medalla obtenida (Gold, Silver, Bronze)
    'record_status',     # Estado del récord (World Record, National Record, etc.)
    'lifting_style'      # Estilo de levantamiento (Olympic, Powerlifting, etc.)
]]

# ============================================================
# RESUMEN FINAL
# ============================================================
print("\n" + "=" * 60)
print("RESUMEN DE fact_lifting:")
print("=" * 60)
print(f"   ✓ Total de registros: {len(fact_lifting)}")
print(f"   ✓ Columnas: {len(fact_lifting.columns)}")
print(f"   ✓ Registros originales: {len(fact_base)}")
print(f"   ✓ Porcentaje procesado: {len(fact_lifting)/len(fact_base)*100:.1f}%")
print(f"   ✓ Columnas en fact_lifting: {list(fact_lifting.columns)}")

print("\n   Muestra de los primeros 5 registros:")
print("-" * 60)
display(fact_lifting.head(5))

print("\n   Información de la tabla:")
print("-" * 60)
print(fact_lifting.info())


## PASO 3: CARGA (L) - Guardar en SQLite

Cargamos todas las tablas en la base de datos SQLite `warehouse_pandas.db`


In [ ]:
# ============================================================
# PASO 3: CARGA (L) - Guardar en SQLite
# ============================================================
print("\n" + "=" * 60)
print("PASO 3: CARGA (L)")
print("=" * 60)

# Crear directorio warehouse si no existe
import os
os.makedirs("warehouse", exist_ok=True)

# Ruta de la base de datos
db_path = "warehouse/warehouse_pandas.db"

# Eliminar base de datos existente si existe (para recrearla)
if os.path.exists(db_path):
    os.remove(db_path)
    print("⚠️ Base de datos existente eliminada")

# Crear conexión a SQLite usando SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}', echo=False)
print(f"✓ Conexión a SQLite establecida: {db_path}")

# Cargar tablas en SQLite usando to_sql()
print("\n📊 Cargando tablas en SQLite...")

# Cargar dim_athlete
dim_athlete.to_sql("dim_athlete", engine, if_exists="replace", index=False)
print(f"   ✓ dim_athlete: {len(dim_athlete)} registros cargados")

# Cargar dim_competition
dim_competition.to_sql("dim_competition", engine, if_exists="replace", index=False)
print(f"   ✓ dim_competition: {len(dim_competition)} registros cargados")

# Cargar dim_team
dim_team.to_sql("dim_team", engine, if_exists="replace", index=False)
print(f"   ✓ dim_team: {len(dim_team)} registros cargados")

# Cargar fact_lifting
fact_lifting.to_sql("fact_lifting", engine, if_exists="replace", index=False)
print(f"   ✓ fact_lifting: {len(fact_lifting)} registros cargados")

print(f"\n✅ Proceso ETL completado. Base de datos guardada en: {db_path}")


In [ ]:
# ============================================================
# VERIFICACIÓN DE DATOS CARGADOS
# ============================================================
print("\n" + "=" * 60)
print("VERIFICACIÓN DE DATOS CARGADOS")
print("=" * 60)

# Verificar datos cargados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consultar número de registros en cada tabla
tables = ["dim_athlete", "dim_competition", "dim_team", "fact_lifting"]
print("\n📊 Resumen de tablas en warehouse_pandas.db:\n")

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"   {table}: {count} registros")

# Mostrar muestras de cada tabla
print("\n" + "-" * 60)
print("MUESTRA DE dim_athlete:")
print("-" * 60)
pd.read_sql_query("SELECT * FROM dim_athlete LIMIT 5", conn)

print("\n" + "-" * 60)
print("MUESTRA DE dim_competition:")
print("-" * 60)
pd.read_sql_query("SELECT * FROM dim_competition LIMIT 5", conn)

print("\n" + "-" * 60)
print("MUESTRA DE dim_team:")
print("-" * 60)
pd.read_sql_query("SELECT * FROM dim_team LIMIT 5", conn)

print("\n" + "-" * 60)
print("MUESTRA DE fact_lifting:")
print("-" * 60)
pd.read_sql_query("SELECT * FROM fact_lifting LIMIT 5", conn)

conn.close()

print("\n" + "=" * 60)
print("¡ETL CON PANDAS COMPLETADO EXITOSAMENTE!")
print("=" * 60)


In [ ]:
# PROYECTO RA1 - FASE 1: Exploración y Limpieza con Pandas

import pandas as pd
import numpy as np

# Cargamos los datos del csv con el read
data_path = "data/gym_lifters.csv"  
df = pd.read_csv(data_path)


print("Dataset cargado correctamente")
print(f"Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
display(df.head())



print("Información general del DataFrame:\n")
df.head()
df.info()

print("Tipos de datos por columna:")
print(df.dtypes)

print("Descripción estadística de columnas numéricas:")
display(df.describe())


print("Valores nulos por columna:")
print(df.isna().sum())

print("Filas duplicadas:", df.duplicated().sum())

# Se eliminan los duplicados si existen
df = df.drop_duplicates()

# Estandarizar nombres de columnas a snake_case
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Eliminar espacios en texto y uniformizar mayúsculas/minúsculas
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip().str.lower()

# Limpiar valores problemáticos antes de convertir a numérico
# Reemplazar valores inválidos comunes por NaN
valores_invalidos = ['?', 'n/a', 'na', 'unknown', '-1', 'none']
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace(valores_invalidos, np.nan)

# Limpiar unidades de texto en columnas numéricas (ej: "154kg" -> 154)
columnas_peso = ['snatch_kg', 'clean_and_jerk_kg', 'total_kg', 'body_weight_kg']
for col in columnas_peso:
    if col in df.columns:
        # Extraer solo números, manejar casos como "154kg", "300kg", etc.
        df[col] = df[col].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convertir columnas numéricas a tipos apropiados
# Age
df["age"] = pd.to_numeric(df["age"], errors='coerce')

# Year (año)
df["year"] = pd.to_numeric(df["year"], errors='coerce')
# Limpiar años inválidos (ej: 9999, años futuros)
df["year"] = df["year"].where((df["year"] >= 1900) & (df["year"] <= 2025), np.nan)

# Event rank
df["event_rank"] = pd.to_numeric(df["event_rank"], errors='coerce')

# Tratar valores faltantes en columnas numéricas
# Age: rellenar con mediana
df["age"] = df["age"].fillna(df["age"].median())

# Pesos: rellenar con mediana de cada columna
for col in columnas_peso:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Year: rellenar con moda (año más común)
df["year"] = df["year"].fillna(df["year"].mode()[0] if not df["year"].mode().empty else np.nan)

# Convertir columnas a tipos de datos adecuados
print("\n[OK] Dataset limpio:")
print(df.info())

print("\n[INFO] Tipos de datos finales:")
print(df.dtypes)

print("\n[INFO] Valores nulos después de limpieza:")
print(df.isna().sum())

print("\n[INFO] Resumen estadístico de columnas numéricas:")
display(df.describe())

# Vista final de los datos
display(df.sample(5))

# Guardar el DataFrame limpio en un nuevo archivo CSV
clean_path = "data/gym_lifters_clean.csv"
df.to_csv(clean_path, index=False)
print(f"Dataset limpio guardado en: {clean_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'gym_lifters.csv'